<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploration-et-nettoyage-des-données" data-toc-modified-id="Exploration-et-nettoyage-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploration et nettoyage des données</a></span><ul class="toc-item"><li><span><a href="#Table-customers" data-toc-modified-id="Table-customers-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Table customers</a></span></li><li><span><a href="#Table-products" data-toc-modified-id="Table-products-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Table products</a></span></li><li><span><a href="#Table-transactions" data-toc-modified-id="Table-transactions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Table transactions</a></span></li></ul></li></ul></div>

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

import re

from P6_functions import *

In [2]:
os.listdir('../data')

['customers.csv',
 'customers_cleaned.pkl',
 'products.csv',
 'products_cleaned.pkl',
 'products_imputed.pkl',
 'pro_clients.pkl',
 'transactions.csv',
 'transactions_cleaned.pkl',
 'txns_prods_merged.pkl']

In [3]:
# on importe les datasets
customers = pd.read_csv('../data/customers.csv')
products = pd.read_csv('../data/products.csv')
transactions = pd.read_csv('../data/transactions.csv')

In [4]:
# on définit le thème visuel des graphiques
sns.set_theme(style='whitegrid', palette='muted')

# Exploration et nettoyage des données

## Table customers

<div class="alert alert-block alert-info">
    <b>Résumé</b><br>
    <ul>
        <li>La table <code>customers</code> contient des informations sur chacun des clients telles que le sexe <code>sex</code> et l'année de naissance <code>birth</code>.</li>
        <li>Les clients sont identifiés par la clé primaire <code>client_id</code>.</li>
        <li>On a calculé l'âge des clients en 2022 à partir de l'année de naissance.</li>
        </ul>
</div>

In [5]:
describe_table(customers, pk='client_id')

La table contient 8623 observations et 3 variables.

Aperçu de la table


,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984



Description des variables


,Compte,Doublons,Valeurs manquantes,Modalités,Type
client_id,8623,0,0,8623,object
sex,8623,8621,0,2,object
birth,8623,8547,0,76,int64



Description des variables numériques


,Min,Max,Moyenne,Médiane
birth,1929,2004,1978.280877,1979.0



Clé primaire
------------
Toutes les valeurs de la clé primaire ['client_id'] sont uniques.
Aucune des valeurs de ['client_id'] n'est manquante.


On modifie les dtypes des variables.

In [6]:
dtype_dict = {
    'client_id': 'string',
    'sex': 'category',
    'birth': 'int32'
}

customers = customers.astype(dtype_dict)
customers.dtypes

client_id      string
sex          category
birth           int32
dtype: object

On calcule l'âge des clients en 2022.

<font color='grey'>Note : Pour simplifier l'analyse, nous avons décidé d'attribuer un âge fixe aux clients. Les transactions s'étalant sur la période de 2021 à 2023, nous avons choisi l'année moyenne 2022.</font>

In [7]:
customers['age'] = 2022 - customers['birth']
customers.head(3)

,client_id,sex,birth,age
0,c_4410,f,1967,55
1,c_7839,f,1975,47
2,c_1699,f,1984,38


## Table products

<div class="alert alert-block alert-info">
    <b>Résumé</b><br>
    <ul>
        <li>La table <code>products</code> contient des informations sur les références telles que le prix <code>price</code> et la catégorie du livre <code>categ</code>.</li>
        <li>Les références sont identifiées par la clé primaire <code>id_prod</code>.
        <li>3287 produits sont référencés.</li>
        <li>Il existe trois catégories de produits différentes (0, 1, 2).</li>
        <li>L'identifiant produit est composé du numéro de la catégorie et d'un identifiant d'un à quatre chiffres.</li>
        <li>On observe une anomalie dans les données : la référence "T_0" a un prix négatif. De plus, la première partie de l'id_prod de cette référence ("T") ne correspond à aucune des trois catégories mentionnées ci-dessus.</li>
        </ul>
</div>

In [9]:
describe_table(products, pk='id_prod')

La table contient 3287 observations et 3 variables.

Aperçu de la table


,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0



Description des variables


,Compte,Doublons,Valeurs manquantes,Modalités,Type
id_prod,3287,0,0,3287,object
price,3287,1832,0,1455,float64
categ,3287,3284,0,3,int64



Description des variables numériques


,Min,Max,Moyenne,Médiane
price,-1.0,300.0,21.856641,13.06
categ,0.0,2.0,0.370246,0.00



Clé primaire
------------
Toutes les valeurs de la clé primaire ['id_prod'] sont uniques.
Aucune des valeurs de ['id_prod'] n'est manquante.


On modifie les dtypes des variables. On convertit la variable `categ` en `string` puis en `category`.

In [10]:
dtype_dict = {
    'id_prod': 'string',
    'price': 'float',
    'categ': 'string',
}

products = products.astype(dtype_dict)
products['categ'] = products['categ'].astype('category')
products.dtypes

id_prod      string
price       float64
categ      category
dtype: object

On examine de plus près les différentes variables à commencer par `id_prod`.

In [11]:
# taille minimale de l'id_prod
products.sort_values(by='id_prod', key=lambda x: x.str.len(), ascending=True).head(1)
# products['id_prod'].apply(len).min()

,id_prod,price,categ
2464,2_3,202.99,2


In [12]:
# taille maximale de l'id_prod
products.sort_values(by='id_prod', key=lambda x: x.str.len(), ascending=False).head(1)

,id_prod,price,categ
0,0_1421,19.99,0


L'identifiant semble être composé de la catégorie du produit suivi d'un numéro d'un à quatre chiffres. On vérifie que le premier caractère correspond toujours à la catégorie du produit.

In [13]:
mask = products['id_prod'].apply(lambda x: x[0]) == products['categ']
mask.value_counts()

True     3286
False       1
dtype: int64

On observe un produit pour lequel ce n'est pas le cas.

In [14]:
products.loc[~mask]

,id_prod,price,categ
731,T_0,-1.0,0


Seule la référence "T_0" ne suit pas cette règle. A noter également que le prix de "T_0" est négatif.

On peut également contrôler cela grâce aux expressions régulières.

In [15]:
id_regex = re.compile(r'[0-2]_\d{1,4}')

In [16]:
mask = products['id_prod'].apply(lambda x: re.fullmatch(id_regex, x)).isnull()
products.loc[mask]

,id_prod,price,categ
731,T_0,-1.0,0


On vérifie si d'autres références ont un prix inférieur ou égal à zéro.

In [17]:
products.loc[products['price'] <= 0]

,id_prod,price,categ
731,T_0,-1.0,0


Le produit "T_0" a un identifiant atypique et un prix négatif. Il pourrait donc constituer une anomalie.

## Table transactions

<div class="alert alert-block alert-info">
    <b>Résumé</b><br>
    <ul>
        <li>La table <code>transactions</code> contient, outre les clés étrangères <code>id_prod</code> et <code>client_id</code>, la date de chaque transaction à la microseconde près (colonne <code>date</code>) et un identifiant de la session (<code>session_id</code>).</li>
        <li>Elle contient 200 transactions tests dont l'identifiant produit est "T_0" (qui correspond à l'anomalie repérée précédemment dans la table <code>products</code>).</li>
        <li>Elle contient 679 332 lignes/transactions après élimination de ces tests.</li>
        <li>Chaque date est unique (une fois les tests éliminés). La colonne <code>date</code> constitue donc la clé primaire de cette table ; une date correspond à une vente.</li>
    </ul>
</div>

On stocke les lignes ayant la même clé primaire dans le dataframe `pk_dups`.

In [18]:
pk_dups, _ = describe_table(transactions, pk=['date', 'session_id'], return_res=True)

La table contient 679532 observations et 4 variables.

Aperçu de la table


,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714



Description des variables


,Compte,Doublons,Valeurs manquantes,Modalités,Type
id_prod,679532,676265,0,3267,object
date,679532,161,0,679371,object
session_id,679532,337216,0,342316,object
client_id,679532,670930,0,8602,object



Clé primaire
------------
198 valeurs de la clé primaire ['date', 'session_id'] ne sont pas uniques.
Aucune des valeurs de ['date', 'session_id'] n'est manquante.


On change les dtypes des variables.

In [19]:
dtype_dict = {
    'id_prod': 'string',
    'session_id': 'string',
    'client_id': 'string'
}

transactions = transactions.astype(dtype_dict)
transactions.dtypes

id_prod       string
date          object
session_id    string
client_id     string
dtype: object

In [20]:
pk_dups.head()

,id_prod,date,session_id,client_id
3019,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
5138,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0
9668,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
10728,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
15292,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0


Les transactions possédant la même clé primaire correspondent à des tests. On va les éliminer du dataset. Cela nous permettra également de convertir la colonne `date` au format `datetime`.

In [21]:
# on sélectionne les lignes dont la date commence par le mot "test"
mask = transactions['date'].str.startswith('test')
n_tests = transactions.loc[mask].shape[0]

print(f"Nombre de transactions tests : {n_tests}")

Nombre de transactions tests : 200


In [22]:
# on affiche pour ces lignes les valeurs uniques des colonnes "id_prod",
# "session_id" et "client_id"
transactions.loc[mask, ['id_prod', 'session_id', 'client_id']].drop_duplicates()

,id_prod,session_id,client_id
3019,T_0,s_0,ct_0
9668,T_0,s_0,ct_1


In [23]:
# on vérifie que ces valeurs ne sont pas utilisées pour d'autres transactions
masks = [
    transactions['id_prod'] == 'T_0',
    transactions['session_id'] == 's_0',
    transactions['client_id'].isin(['ct_0', 'ct_1'])
]

for mask in masks:
    print(transactions.loc[mask].shape[0])

200
200
200


Ces valeurs n'ont été utilisées que pour les tests. On peut supprimer ces derniers de la table.

In [24]:
# on élimine les lignes correspondant aux tests
mask = transactions['id_prod'] != 'T_0'
transactions = transactions.loc[mask]

On peut également supprimer les clients fictifs "ct_0" et "ct_1" de la table `customers` et le produit "T_0" de la table `products`.

In [25]:
n_rows_before = customers.shape[0]
customers = customers.loc[~(customers['client_id'].isin(['ct_0', 'ct_1']))]
n_rows_after = customers.shape[0]
n_deleted = n_rows_before - n_rows_after

print(f"Nombre de lignes supprimées de la table `customers` : {n_deleted}")

Nombre de lignes supprimées de la table `customers` : 2


In [26]:
n_rows_before = products.shape[0]
products = products.loc[products['id_prod'] != 'T_0']
n_rows_after = products.shape[0]
n_deleted = n_rows_before - n_rows_after

print(f"Nombre de lignes supprimées de la table `products` : {n_deleted}")

Nombre de lignes supprimées de la table `products` : 1


In [27]:
describe_table(transactions, pk=['date', 'session_id'])

La table contient 679332 observations et 4 variables.

Aperçu de la table


,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714



Description des variables


,Compte,Doublons,Valeurs manquantes,Modalités,Type
id_prod,679332,676066,0,3266,string
date,679332,0,0,679332,object
session_id,679332,337017,0,342315,string
client_id,679332,670732,0,8600,string



Clé primaire
------------
Toutes les valeurs de la clé primaire ['date', 'session_id'] sont uniques.
Aucune des valeurs de ['date', 'session_id'] n'est manquante.


<div class="alert alert-block alert-info">
    <b>Note</b><br>
    Plutôt que d'utiliser une clé primaire composée, on va créer une clé artificielle. A noter que toutes les valeurs de <code>date</code> étant uniques, on pourrait utiliser cette variable comme clé primaire. Cependant, cela ne nous semble pas être une bonne pratique car on ne peut pas écarter la possibilité que deux achats soient effectués en même temps (à la microseconde près) à l'avenir.
</div>

In [28]:
# on crée une clé artificielle
transactions['id_txn'] = np.arange(transactions.shape[0])
transactions['id_txn'] = 't_' + transactions['id_txn'].astype('string')
transactions['id_txn'] = transactions['id_txn'].astype('string')
transactions.head(3)

,id_prod,date,session_id,client_id,id_txn
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,t_0
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,t_1
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,t_2


On peut maintenant changer le dtype de `date` en `datetime`.

In [29]:
transactions['date'] = pd.to_datetime(transactions['date'])

On extrait le jour de chaque transaction à partir de la date.

In [30]:
transactions['day'] = transactions['date'].dt.strftime('%Y-%m-%d')
transactions['day'] = pd.to_datetime(transactions['day'])

In [31]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679531
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  string        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  string        
 3   client_id   679332 non-null  string        
 4   id_txn      679332 non-null  string        
 5   day         679332 non-null  datetime64[ns]
dtypes: datetime64[ns](2), string(4)
memory usage: 36.3 MB


In [32]:
# on pickle les dataframes pour conserver les dtypes
customers.to_pickle('../data/customers_cleaned.pkl')
products.to_pickle('../data/products_cleaned.pkl')
transactions.to_pickle('../data/transactions_cleaned.pkl')

In [33]:
transactions_cleaned = pd.read_pickle('../data/transactions_cleaned.pkl')

In [34]:
# on vérifie que les dtypes ont bien été conservés après import
transactions_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679531
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  string        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  string        
 3   client_id   679332 non-null  string        
 4   id_txn      679332 non-null  string        
 5   day         679332 non-null  datetime64[ns]
dtypes: datetime64[ns](2), string(4)
memory usage: 36.3 MB
